# Pipelines

Pipelines are an integral part of River. We encourage their usage and apply them in many of their examples.

The `compose.Pipeline` contains all the logic for building and applying pipelines. A pipeline is essentially a list of estimators that are applied in sequence. The only requirement is that the first `n - 1` steps be transformers. The last step can be a regressor, a classifier, a clusterer, etc.

Here is an example:

In [80]:
from river import compose
from river import linear_model
from river import preprocessing
from river import feature_extraction

model = compose.Pipeline(
    preprocessing.StandardScaler(),
    feature_extraction.PolynomialExtender(),
    linear_model.LinearRegression()
)

You can also use the `|` operator, as so:

In [81]:
model = (
    preprocessing.StandardScaler() |
    feature_extraction.PolynomialExtender() |
    linear_model.LinearRegression()
)

Or, equally:

In [82]:
model = preprocessing.StandardScaler() 
model |= feature_extraction.PolynomialExtender()
model |= linear_model.LinearRegression()

A pipeline has a `_repr_html_` method, which can be used to visualize it:

In [83]:
model

Pipeline (
  StandardScaler (
    with_std=True
  ),
  PolynomialExtender (
    degree=2
    interaction_only=False
    include_bias=False
    bias_name="bias"
  ),
  LinearRegression (
    optimizer=SGD (
      lr=Constant (
        learning_rate=0.01
      )
    )
    loss=Squared ()
    l2=0.
    l1=0.
    intercept_init=0.
    intercept_lr=Constant (
      learning_rate=0.01
    )
    clip_gradient=1e+12
    initializer=Zeros ()
  )
)

`compose.Pipeline` implements a `learn_one` method which in sequence calls the `learn_one` of each step and a `predict_one` (resp `predict_proba_one`) method which calls `transform_one` on the first `n - 1` steps and `predict_one` (resp `predict_proba_one`) on the last step.

Here is a small example to illustrate the previous point:

In [84]:
from river import datasets

dataset = iter(datasets.TrumpApproval())
x, y = next(dataset)
x, y

({'ordinal_date': 736389,
  'gallup': 43.843213,
  'ipsos': 46.19925042857143,
  'morning_consult': 48.318749,
  'rasmussen': 44.104692,
  'you_gov': 43.636914000000004},
 43.75505)

In [85]:
model.learn_one(x, y)

x, y = next(dataset)
model.learn_one(x, y)

x, _ = next(dataset)
model.predict_one(x)

28.28960916

Each component/step of the pipeline has been updated with the new data point. 

A pipeline is a very powerful tool that can be used to chain together multiple steps in a machine learning workflow.

Notice that it is also possible to call `transform_one` with a pipeline, this method will run `transform_one` of each transformer in it, and return the result of the last transformer (which is thus the penultimate step):

In [86]:
model.transform_one(x)

{'ordinal_date': 3.0,
 'gallup': 0.0,
 'ipsos': 0.0,
 'morning_consult': 0.0,
 'rasmussen': 2.0,
 'you_gov': -1.0,
 'ordinal_date*ordinal_date': 9.0,
 'gallup*ordinal_date': 0.0,
 'ipsos*ordinal_date': 0.0,
 'morning_consult*ordinal_date': 0.0,
 'ordinal_date*rasmussen': 6.0,
 'ordinal_date*you_gov': -3.0,
 'gallup*gallup': 0.0,
 'gallup*ipsos': 0.0,
 'gallup*morning_consult': 0.0,
 'gallup*rasmussen': 0.0,
 'gallup*you_gov': -0.0,
 'ipsos*ipsos': 0.0,
 'ipsos*morning_consult': 0.0,
 'ipsos*rasmussen': 0.0,
 'ipsos*you_gov': -0.0,
 'morning_consult*morning_consult': 0.0,
 'morning_consult*rasmussen': 0.0,
 'morning_consult*you_gov': -0.0,
 'rasmussen*rasmussen': 4.0,
 'rasmussen*you_gov': -2.0,
 'you_gov*you_gov': 1.0}

In many cases, you might want to connect a step to multiple steps. For instance, you might to extract different kinds of features from a single input. An elegant way to do this is to use a `compose.TransformerUnion`. Essentially, the latter is a list of transformers who's results will be merged into a single `dict` when `transform_one` is called.

As an example let's say that we want to apply a `feature_extraction.RBFSampler` as well as the `feature_extraction.PolynomialExtender`. This may be done as so:

In [87]:
model = (
    preprocessing.StandardScaler() |
    (feature_extraction.PolynomialExtender() + feature_extraction.RBFSampler()) |
    linear_model.LinearRegression()
)

model

Pipeline (
  StandardScaler (
    with_std=True
  ),
  TransformerUnion (
    PolynomialExtender (
      degree=2
      interaction_only=False
      include_bias=False
      bias_name="bias"
    ),
    RBFSampler (
      gamma=1.
      n_components=100
      seed=None
    )
  ),
  LinearRegression (
    optimizer=SGD (
      lr=Constant (
        learning_rate=0.01
      )
    )
    loss=Squared ()
    l2=0.
    l1=0.
    intercept_init=0.
    intercept_lr=Constant (
      learning_rate=0.01
    )
    clip_gradient=1e+12
    initializer=Zeros ()
  )
)

Note that the `+` symbol acts as a shorthand notation for creating a `compose.TransformerUnion`, which means that we could have declared the above pipeline as so:

In [88]:
model = (
    preprocessing.StandardScaler() |
    compose.TransformerUnion(
        feature_extraction.PolynomialExtender(),
        feature_extraction.RBFSampler()
    ) |
    linear_model.LinearRegression()
)

Pipelines provide the benefit of removing a lot of cruft by taking care of tedious details for you. They also enable to clearly define what steps your model is made of.

Finally, having your model in a single object means that you can move it around more easily.

Note that you can include user-defined functions in a pipeline by using a `compose.FuncTransformer`.

## Learning during predict

In online macine learning, we can update the unsupervised parts of our model when a sample arrives. We don't _really_ have to wait for the ground truth to arrive in order to update unsupervised estimators that don't depend on it.

In other words, in a pipeline, `learn_one` updates the supervised parts, whilst `predict_one` (or `predict_proba_one` for that matter) **can** update the unsupervised parts. 

In river, we can achieve this behavior using a dedicated context manager: `compose.learn_during_predict`.

Here is the same example as before, with the only difference of activating the such learning during predict behavior:

In [89]:
model = (
    preprocessing.StandardScaler() |
    feature_extraction.PolynomialExtender() |
    linear_model.LinearRegression()
)

In [90]:
with compose.learn_during_predict():

    x, y = next(dataset)
    model.predict_one(x)

    x, y = next(dataset)
    model.predict_one(x)

model['StandardScaler'].means, model['StandardScaler'].vars 

(defaultdict(float,
             {'ordinal_date': 736392.5,
              'gallup': 43.843213,
              'ipsos': 46.437345666666666,
              'morning_consult': 48.318749,
              'rasmussen': 47.104692,
              'you_gov': 41.636914000000004}),
 defaultdict(float,
             {'ordinal_date': 0.25,
              'gallup': 0.0,
              'ipsos': 0.0,
              'morning_consult': 0.0,
              'rasmussen': 0.0,
              'you_gov': 0.0}))

Calling `predict_one` within this context will update each transformer of the pipeline. For instance here we can see that the mean and the variance each feature of the standard scaler step have been updated.

On the other hand, the supervised part of our pipeline, the linear regression, has not been updated or learned anything yet. Hence the prediction on any sample will be nil because each weight is still equal to 0.

In [91]:
model.predict_one(x), model["LinearRegression"].weights

(0.0, {})